In [3]:
import requests
import json
import dml
import prov.model
import datetime
import uuid
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [4]:
contributor = 'mmao95_Dongyihe_weijiang_zhukk'
reads = []
writes = [contributor + '.famous_people']

## exec

In [6]:
startTime = datetime.datetime.now()

# Set up the database connection.
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

True

In [15]:
url = 'http://datamechanics.io/data/us_famous_people.csv'
usfp = pd.read_csv(url)
names = usfp['full_name'].values
len(names)

110

In [21]:
doc = requests.get("https://www.50states.com/bio/mass.htm").text
soup = bs(doc)
names = np.append(names, [e.text for e in soup.select("#content b")])
len(names)

230

In [22]:
list = []
for name in names:
    parts = name.split()
    obj = {
        'full_name': name,
        'first_name': parts[0],
        'last_name': parts[-1] if len(parts) > 1 else None,
        'middle_name': parts[1:-1] if len(parts) > 2 else None
    }
    list.append(obj)

In [23]:
repo.dropCollection('famous_people')
repo.createCollection('famous_people')
repo[writes[0]].insert_many(list)

In [24]:
repo[writes[0]].metadata({'complete': True})
print(repo[writes[0]].metadata())
repo[writes[0]].count_documents({})


{'_id': ObjectId('5ca3734299d98669f193b988'), 'complete': True}


230

In [25]:
repo.logout()
endTime = datetime.datetime.now()
{"start":startTime, "end":endTime}

/home/dongguho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Database.logout() is deprecated
  """Entry point for launching an IPython kernel.


{'start': datetime.datetime(2019, 4, 2, 22, 30, 17, 192069),
 'end': datetime.datetime(2019, 4, 2, 22, 35, 52, 550110)}

## prov

In [9]:
doc = prov.model.ProvDocument()
startTime = None
endTime = None
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate(contributor, contributor)

doc.add_namespace('alg', 'http://datamechanics.io/algorithm/') # The scripts are in <folder>#<filename> format.
doc.add_namespace('dat', 'http://datamechanics.io/data/') # The data sets are in <user>#<collection> format.
doc.add_namespace('ont', 'http://datamechanics.io/ontology#') # 'Extension', 'DataResource', 'DataSet', 'Retrieval', 'Query', or 'Computation'.
doc.add_namespace('log', 'http://datamechanics.io/log/') # The event log.
doc.add_namespace('bdp', 'https://www.50states.com/bio/mass.htm')

this_script = doc.agent('alg:'+contributor+'#famous_people', {prov.model.PROV_TYPE:prov.model.PROV['SoftwareAgent'], 'ont:Extension':'py'})
resource = doc.entity('bdp:wc8w-nujj', {'prov:label':'311, Service Requests', prov.model.PROV_TYPE:'ont:DataResource', 'ont:Extension':'json'})
get_names = doc.activity('log:uuid'+str(uuid.uuid4()), startTime, endTime)
doc.wasAssociatedWith(get_names, this_script)
doc.usage(get_names, resource, startTime, None,
    {prov.model.PROV_TYPE:'ont:Retrieval',
    'ont:Computation':'Data cleaning'
    }
)

fp = doc.entity('dat:'+contributor+'#famous_people', {prov.model.PROV_LABEL:'Famous People', prov.model.PROV_TYPE:'ont:DataSet'})
doc.wasAttributedTo(fp, this_script)
doc.wasGeneratedBy(fp, get_names, endTime)
doc.wasDerivedFrom(fp, resource, get_names, get_names, get_names)

repo.logout()
doc

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Database.logout() is deprecated


<ProvDocument>

In [10]:
print(doc.get_provn())
print(json.dumps(json.loads(doc.serialize()), indent=4))

document
  prefix alg <http://datamechanics.io/algorithm/>
  prefix dat <http://datamechanics.io/data/>
  prefix ont <http://datamechanics.io/ontology#>
  prefix log <http://datamechanics.io/log/>
  prefix bdp <https://www.50states.com/bio/mass.htm>
  
  agent(alg:mmao95_Dongyihe_weijiang_zhukk#famous_people, [prov:type='prov:SoftwareAgent', ont:Extension="py"])
  entity(bdp:wc8w-nujj, [prov:label="311, Service Requests", prov:type="ont:DataResource", ont:Extension="json"])
  activity(log:uuide2575f89-f744-4fe5-9902-d1d92e7034fb, -, -)
  wasAssociatedWith(log:uuide2575f89-f744-4fe5-9902-d1d92e7034fb, alg:mmao95_Dongyihe_weijiang_zhukk#famous_people, -)
  used(log:uuide2575f89-f744-4fe5-9902-d1d92e7034fb, bdp:wc8w-nujj, -, [prov:type="ont:Retrieval", ont:Computation="Data cleaning"])
  entity(dat:mmao95_Dongyihe_weijiang_zhukk#famous_people, [prov:label="Famous People", prov:type="ont:DataSet"])
  wasAttributedTo(dat:mmao95_Dongyihe_weijiang_zhukk#famous_people, alg:mmao95_Dongyihe_weij